In [1]:
def SpectrogramBandDivisor(spectrogram,n_mels):
    
    """
    This function will :
        1) Transpose the spectrogram
        2) Takes #frames bands of the spectrogram and concatenate them
    
    """
    timestamps = [0.2, 0.4, 0.5, 0.6, 0.8] 
    
    time_index = [round(timestamp*spectrogram.shape[1]) for timestamp in timestamps]
    
    res_list = spectrogram[:,time_index[0]]
    for t in range(len(timestamps)-1):
        res_list = [*res_list, *spectrogram[:,time_index[t+1]]]
    return res_list

In [2]:
def SpectrogramDuplicator(spectrogram, n_mels, frames=5):
    """
    1) Transpose the spectrogram
    2) Duplicate it by #frames
    
    """
    
    dims = frames * n_mels
    vectorarray_size = len(spectrogram[0, :]) - frames + 1

    vectorarray = np.zeros((vectorarray_size, dims), float)
    for t in range(frames):
           vectorarray[:, n_mels * t: n_mels * (t + 1)] = spectrogram[:, t: t + vectorarray_size].T
    return vectorarray       
           
           
def import_data(wav_name,channel=0):
    try:
        multi_channel_data, sr = sf.read(wav_name)
        if multi_channel_data.ndim <= 1:
            # ADD HERE A CONDITION TO READ MULTICHANNEL LENGTH AND REMOVE IF TOO SHORT
            return sr, multi_channel_data
        return sr, np.array(multi_channel_data)[:,channel]
    except ValueError as msg:
        logger.warning(f'{msg}')

           
def Spectro_DataFrame_Maker(user_path,data_path, SpectroDuplication = False, SpectroBandDivisor = True):
    # The data_path must be CSV files of
    # 1) all same length
    # 2) audio data
    your_path = user_path+data_path
    files = os.listdir(your_path)    
    
    """
    Spectrogram parameters
    """
    n_mels = 64
    n_fft = 1024
    overlap = round(n_fft*0.5)
    power = 2
    
    
    S_dB_list = []
    for index,file in enumerate(files):
        if os.path.isfile(os.path.join(your_path,file)):
            fs, audio = import_data(os.path.join(your_path,file))
            S = librosa.feature.melspectrogram(y=audio,
                                                     sr=fs,
                                                     n_mels = n_mels,
                                                     #fmax=8000   default is sr/2
                                                     n_fft= n_fft,
                                                     hop_length= overlap,
                                                     power= power                                    )
            S_dB = librosa.power_to_db(S)
            
            if SpectroDuplication == True:
                S_dB = SpectrogramDuplicator(S_dB,n_mels)
                S_dB_flat = S_dB.flatten()  # flatten the matrix to 1D vector (raw-wise)
            if SpectroBandDivisor == True:
                S_dB_flat = SpectrogramBandDivisor(S_dB,n_mels) # By default, duplication = 5
            S_dB_list.append(S_dB_flat)
    return pd.DataFrame(S_dB_list)

In [3]:
import numpy as np
import os 
import librosa
import librosa.display
import soundfile as sf
import pandas as pd

    
names = ['fan']
IDs = ['id_00']
stati = ['normal']
    
folder = 'MIMII/RawData/' # fan/id_00/abnormal'
user_path = 'C:/Users/carbo/Documents/'
writer_path = 'C:/Users/carbo/Documents/MIMII/CSV_Data/Spectrogram_5Bands/'
for name in names:
    for ID in IDs:
        for status in stati:
            data_path = folder + name + '/' + ID + '/' + status
            df_spectro = Spectro_DataFrame_Maker(user_path,data_path)

In [5]:
df_spectro.shape

(1011, 320)